# I. Business Context & Project Background 

## 1.1 What Is the Case?


&emsp;Forecasts are indispensable for many of the daily decisions that we make, such as time to get up in the morning in order to not be late for work, or when is the best time to fill up the tank before the gas price goes up. Supermarkets require forecasts to support their strategic development, make tactical decisions, and manage their demand and supply planning processes in order to avoid customer service issues and high inventory costs (Fildes et al., 2019). Therefore, it is important for supermarkets like Walmart to forecast as accurately as possible because stocking too many products incurs extra costs whereas stocking insufficient would lead to lost sales and low profits. The M competitions have been conducted for almost 40 years , and they aim to identify ways to improve the accuracy of forecasting by empirically evaluating several forecasting methods and identifying the one with the best accuracy. The findings obtained in these competitions have significantly influenced the theory and practice of forecasting by providing valuable insights into how the forecasting accuracy can be improved (Hyndman, 2020).
 
&emsp;The M5 competition extended the objectives of the previous four competitions by focusing on a retail sales forecasting application and using real-life, hierarchically structured sales data with intermittent and erratic characteristics (Syntetos and Boylan, 2005, Syntetos et al., 2005). The competition attracted many participants who were eager to experiment with effective forecasting solutions in the real-life situation faced by numerous retail companies on a daily basis. 


## 1.2 What Is Success?

&emsp; The objective of the M5 “Accuracy” competition is to forecast Walmart daily sales in 10 stores from 3 states for the next 28 days (4 weeks ahead) for each 42,840 units. For prediction performance measure, the case competition utilized a variant of the MASE originally proposed by Hyndman and Koehler (2006) called the Root Mean Squared Scaled Error (RMSSE) to measure “Point forecasts”. The measure is calculated for each series as follows:

$$
  RMSSE = \sqrt{\frac{1}{h}  \frac{\sum\limits_{t=n+1}^{n+h}(Y_t-\hat{Y_t})^2} {\frac{1}{n-1} \sum\limits_{t=2}^{n}(Y_t-Y_{t-1})^2} }
$$

After estimating the RMSSE for all the 42,840 time series of the competition, the participating methods will be ranked using the Weighted RMSSE (WRMSSE), using the following formula to determine the top 5 of most accurate point forecasts participants:


$$
WRMSSE = \sum\limits_{i=1}^{42,840} w_i * RMSSE
$$

From the business’s perspective, if we can develop a predictive algorithm that could predict daily sales with high accuracy in the real world retail industry setting, the supermarkets like Walmart will be able to make better decisions on its ordering from suppliers, inventory level, transportation planning, as well as staffing schedules in the future and that could not only maximize its profit but also to improve both customers and employees satisfaction.


# II. Technical Specifications & Solution Overview

## 2.1 Which Tools are Used?

The main machine learning algorithm used in this project is LightGBM. It is a gradient boosting framework based on decision trees to increase the efficiency of the model and reduce memory usage. 

The other packages used for data cleaning, model building, training, testing, and validation are as follow: 
Pandas (v 0.24.0)
Numpy (v 1.16.2), Tensorflow (v 2.0.0) Sklearn (v 0.20.3)


## 2.2 Which Datasets are Used

&emsp; All data used for this analysis were provided by the Kaggle M5 competition in the CSV format. The dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments. The products are sold across ten stores, located in three States (CA, TX, and WI) in the USA. For the time series, the historical data range from <code>2011-01-29</code> to <code>2016-06-19</code>. Thus, the products have a (maximum) selling history of 1,941 days (5.4 years). The three main data files are described below.

* <code>calendar.csv</code> - Contains the dates on which products are sold. The dates are in a <code>yyyy/dd/mm</code> format.

* <code>sales_train_validation.csv</code> - Contains the historical daily unit sales data per product and store <code>[d_1 - d_1913]</code>.

* <code>submission.csv</code> - Demonstrates the correct format for submission to the competition.

* <code>sell_prices.csv</code> - Contains information about the price of the products sold per store and date.

* <code>sales_train_evaluation.csv</code> - Available one month before the competition deadline. It will include sales for <code>[d_1 - d_1941]</code>.


## 2.3 Exploratory data analysis


&emsp; The first step to analyzing the problem is understanding the situation, complication, and key question. Before beginning the modeling process, the data needs some processing as the data fed into the algorithm is important to ensure an accurate result.

&emsp;For that reason, in the steps described in the section below, we perform data transformations and feature engineering to extract the relevant features that have strong predicting power. Next, we dealt with outliers as there were many outliers in the datasets. Finally, we utilized two different models to generate final predictions, a LightGBM and an XGBoost and ensembled them using a neural network as a metamodel.


# III. Predictive Model

To solve the forecasting problem, we first try both neural network technique: LSTM and greadient boosting technique: lightGBM. But the performance of LSTM was not as good as we expected, and due to the computation resources limitation, and the implementation complexity, we decided to focus only on lightGBM.


## 3.1 Model structure

our features can be categorized into the following 3 different parts:
> 1. First is sales related features, like lag sales for different time period, and rolling sales for different time period. This type of features reflect trend component.
> 2. Second is price related features, like max, min standard deviation of prices for that product.
> 3. The last one is calendar related features. Different weekdays like Monday, Tuesday, Wednesday; different month like January, February and so on and so forth. The reason why we created those features is that we blieved based on seasonality, each different time zone has has a certain characteristic or type of behavior.

Based on those feature we will use different models for different weeks, we originally treat 4 weeks as a whole unit and directly predict the whole month. But the performance isn’t good enough, so after a couple of thoughts, we realized that  each week has its own characteristics, intuitively saying, when we take about time schedule, human being will treat a whole week independently, so four weeks will have four different situations inherently. 

And in each week we will have 10 different models for different stores, so we will have 4 week* 10stores = 40 models in total. The reason why we use different models for different stores is that, according to our exploratory analysis, we observed that different stores has its own patterns, which intuitively makes sense, since people in different area has their own buying habbits, like stores in California (CA) sells more items in general, while Wisconsin (WI) was slowly catching up to Texas (TX) 

# 